In [ ]:
import tensorflow as tf
import numpy as np
import samples as ls
import model_types

load_samples = False
model_num = 101

if model_num == 0:
    input_dim = 6
    input_steps = 20
    hidden_layers = [50,10]
    labels,sample_ids,filename = model_types.get_labels(0)
elif model_num == 1:
    input_dim = 6
    input_steps = 20
    hidden_layers = [50,10]
    labels,sample_ids,filename = model_types.get_labels(1)
elif model_num == 10:
    input_dim = 6
    input_steps = 50
    hidden_layers = [50,10]
    labels,sample_ids,filename = model_types.get_labels(10)
elif model_num == 100:
    input_dim = 6
    input_steps = 50
    hidden_layers = [50,10]
    labels,sample_ids,filename = model_types.get_labels(11)
elif model_num == 101:
    input_dim = 6
    input_steps = 50
    hidden_layers = [50]
    labels,sample_ids,filename = model_types.get_labels(11)
else:
    raise ValueError('Unknown model num')
    
classes = len(labels)

#percentage used for test data
test_rate = 0.1

learning_rate = 0.01

In [ ]:
if load_samples:
    samples = ls.load(filename)
else:
    samples = ls.Samples(labels)

    for label in labels:
        samples.load_samples(label = label, sample_ids = sample_ids)
    
    samples.convert_to_input(size = None, accel='xyz', gyro='xyz', compass='')
    
samples.unroll(input_steps)
samples.split_test(test_rate)
samples.flatten()

In [ ]:
tf.logging.set_verbosity(tf.logging.ERROR)

#x,y instead of input_fn
#padding is set to false, since all samples already have to have the same length 
#(if not, unroll was called with to big of a step size)
xtrain_all,ytrain_all,_ = samples.get_all(padding=False)
xtrain_all = np.array(xtrain_all)
ytrain_all = np.array(ytrain_all)
xtest,ytest,_ = samples.get_test(padding=False)
xtest = np.array(xtest)
ytest = np.array(ytest)

validation_monitor = tf.contrib.learn.monitors.ValidationMonitor(
    xtest,
    ytest,
    every_n_steps=1)

feature_columns = [tf.contrib.layers.real_valued_column(
    "", 
    dimension=input_dim*input_steps)]
classifier = tf.contrib.learn.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=hidden_layers,
    n_classes=classes,
    model_dir='tmp/dnn/model-%s' % model_num,
    config=tf.contrib.learn.RunConfig(save_checkpoints_steps=1, save_checkpoints_secs=None)
    )

    

In [ ]:
def train(epochs, steps, batch_size):

    for i in range(epochs):
        xtrain,ytrain,_ = samples.get_batch(batch_size, padding=False)
        xtrain = np.array(xtrain)
        ytrain = np.array(ytrain)
        
        classifier.fit(x=xtrain, y=ytrain, steps=steps, monitors=[validation_monitor])
        accuracy = classifier.evaluate(x=xtest, y=ytest)
        print('Progress: %s' % (float(i)/epochs))
        print('Accuracy: %s' % accuracy["accuracy"])
    accuracy = classifier.evaluate(x=xtrain_all, y=ytrain_all)
    print('Accuracy all: %s' % accuracy["accuracy"])

def per_label_acc():
    accs = [0] * classes
    counts = [0] * classes

    for i in range(len(ytest)):
        y = ytest[i]
        x = xtest[i]
        accuracy = classifier.evaluate(x=np.array([x]), y=np.array([y]))

        accs[y] += accuracy['accuracy']
        counts[y] += 1

    for i in range(classes):
        acc = accs[i]/counts[i]
        print('%s: %s' % (labels[i],acc))

In [ ]:
if True:
    train(1000,10,20)

In [ ]:
if True:
    per_label_acc()

In [ ]:
def feed_data(data):
    res = classifier.predict(x=np.array([data]), as_iterable=False)
    return res

In [ ]:
#live demo
%env ROS_HOSTNAME=zinunb
%env ROS_IP=$(hostname -I)
%env ROS_MASTER_URI=http://pi:11311

if False:
    import imu_listener as il
    reset()
    il.imu_listener(feed_data,labels,input_steps)
    print('Exited')